# Esercitazione Pratica: Algoritmi di Clustering

## Corso di Machine Learning: Apprendimento Non Supervisionato

Questa esercitazione pratica ti guiderà attraverso l'implementazione e l'applicazione dei principali algoritmi di clustering discussi nelle lezioni teoriche. Esploreremo diversi dataset e vedremo come applicare e valutare vari metodi di clustering.

### Obiettivi dell'esercitazione:
- Implementare e applicare K-means, Clustering Gerarchico e DBSCAN
- Visualizzare i risultati del clustering
- Valutare la qualità dei cluster ottenuti
- Confrontare le prestazioni dei diversi algoritmi
- Applicare il clustering a un caso di studio reale: segmentazione dei clienti

## Configurazione dell'ambiente

Iniziamo importando le librerie necessarie per questa esercitazione.

In [ ]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')

# Impostazioni di visualizzazione
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Parte 1: Generazione e Visualizzazione dei Dataset

Prima di applicare gli algoritmi di clustering, generiamo alcuni dataset sintetici con diverse strutture per testare i vari algoritmi.

In [ ]:
# Funzione per visualizzare i dataset
def plot_dataset(X, y=None, title="Dataset"):
    plt.figure(figsize=(10, 6))
    if y is not None:
        plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=50, alpha=0.8)
    else:
        plt.scatter(X[:, 0], X[:, 1], s=50, alpha=0.8)
    plt.title(title, fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.colorbar(label='Cluster' if y is not None else None)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Dataset con cluster ben separati (ideale per K-means)
X_blobs, y_blobs = make_blobs(n_samples=300, centers=4, cluster_std=0.6, random_state=42)
plot_dataset(X_blobs, y_blobs, "Dataset 1: Cluster ben separati")

# 2. Dataset con cluster non sferici (sfida per K-means)
X_moons, y_moons = make_moons(n_samples=300, noise=0.08, random_state=42)
plot_dataset(X_moons, y_moons, "Dataset 2: Cluster a forma di mezzaluna")

# 3. Dataset con cluster concentrici (sfida per molti algoritmi)
X_circles, y_circles = make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)
plot_dataset(X_circles, y_circles, "Dataset 3: Cluster concentrici")

# 4. Dataset con densità variabile (ideale per DBSCAN)
X_varied = np.vstack([
    np.random.normal(0, 0.5, (100, 2)),  # Cluster denso
    np.random.normal(5, 2, (200, 2))     # Cluster sparso
])
y_varied = np.hstack([np.zeros(100), np.ones(200)])
plot_dataset(X_varied, y_varied, "Dataset 4: Cluster con densità variabile")

## Parte 2: K-means Clustering

Implementiamo e applichiamo l'algoritmo K-means ai nostri dataset.

### 2.1 Determinare il numero ottimale di cluster con il metodo del gomito

In [ ]:
def plot_elbow_method(X, max_k=10):
    distortions = []
    silhouette_scores = []
    K = range(2, max_k+1)
    
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)
        
        # Calcolo del silhouette score
        if k > 1:  # Silhouette score richiede almeno 2 cluster
            labels = kmeans.labels_
            silhouette_scores.append(silhouette_score(X, labels))
    
    # Plot del metodo del gomito
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(K, distortions, 'bo-')
    plt.xlabel('Numero di cluster (k)')
    plt.ylabel('Distorsione (Inertia)')
    plt.title('Metodo del gomito')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(K[1:], silhouette_scores, 'ro-')
    plt.xlabel('Numero di cluster (k)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score per diversi valori di k')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return distortions, silhouette_scores

In [ ]:
# Applicare il metodo del gomito al dataset con cluster ben separati
distortions_blobs, silhouette_blobs = plot_elbow_method(X_blobs, max_k=10)

### 2.2 Implementazione di K-means

In [ ]:
def apply_kmeans(X, n_clusters=4, title="K-means Clustering"):
    # Applicare K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    y_pred = kmeans.fit_predict(X)
    centers = kmeans.cluster_centers_
    
    # Visualizzare i risultati
    plt.figure(figsize=(10, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap='viridis', s=50, alpha=0.8)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200, alpha=1, label='Centroidi')
    plt.title(title, fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.colorbar(label='Cluster')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calcolare metriche di valutazione
    if n_clusters > 1:  # Silhouette score richiede almeno 2 cluster
        sil_score = silhouette_score(X, y_pred)
        ch_score = calinski_harabasz_score(X, y_pred)
        db_score = davies_bouldin_score(X, y_pred)
        
        print(f"Silhouette Score: {sil_score:.3f} (più alto è meglio)")
        print(f"Calinski-Harabasz Index: {ch_score:.3f} (più alto è meglio)")
        print(f"Davies-Bouldin Index: {db_score:.3f} (più basso è meglio)")
    
    return y_pred, centers

In [ ]:
# Applicare K-means al dataset con cluster ben separati
y_pred_blobs, centers_blobs = apply_kmeans(X_blobs, n_clusters=4, title="K-means su cluster ben separati")

In [ ]:
# Applicare K-means al dataset con cluster a forma di mezzaluna
y_pred_moons, centers_moons = apply_kmeans(X_moons, n_clusters=2, title="K-means su cluster a forma di mezzaluna")

In [ ]:
# Applicare K-means al dataset con cluster concentrici
y_pred_circles, centers_circles = apply_kmeans(X_circles, n_clusters=2, title="K-means su cluster concentrici")

### 2.3 K-means++: Miglioramento dell'inizializzazione

K-means++ è l'algoritmo di inizializzazione predefinito in scikit-learn. Vediamo come funziona e confrontiamolo con l'inizializzazione casuale.

In [ ]:
def compare_kmeans_init(X, n_clusters=4):
    # K-means con inizializzazione casuale
    kmeans_random = KMeans(n_clusters=n_clusters, init='random', random_state=42, n_init=10)
    y_pred_random = kmeans_random.fit_predict(X)
    inertia_random = kmeans_random.inertia_
    
    # K-means con inizializzazione k-means++
    kmeans_plus = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42, n_init=10)
    y_pred_plus = kmeans_plus.fit_predict(X)
    inertia_plus = kmeans_plus.inertia_
    
    # Visualizzare i risultati
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.scatter(X[:, 0], X[:, 1], c=y_pred_random, cmap='viridis', s=50, alpha=0.8)
    plt.scatter(kmeans_random.cluster_centers_[:, 0], kmeans_random.cluster_centers_[:, 1], 
                c='red', marker='X', s=200, alpha=1, label='Centroidi')
    plt.title(f"K-means con inizializzazione casuale\nInertia: {inertia_random:.2f}", fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.scatter(X[:, 0], X[:, 1], c=y_pred_plus, cmap='viridis', s=50, alpha=0.8)
    plt.scatter(kmeans_plus.cluster_centers_[:, 0], kmeans_plus.cluster_centers_[:, 1], 
                c='red', marker='X', s=200, alpha=1, label='Centroidi')
    plt.title(f"K-means con inizializzazione k-means++\nInertia: {inertia_plus:.2f}", fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Confrontare le metriche
    sil_random = silhouette_score(X, y_pred_random)
    sil_plus = silhouette_score(X, y_pred_plus)
    
    print(f"Inizializzazione casuale - Inertia: {inertia_random:.2f}, Silhouette: {sil_random:.3f}")
    print(f"Inizializzazione k-means++ - Inertia: {inertia_plus:.2f}, Silhouette: {sil_plus:.3f}")
    
    return kmeans_random, kmeans_plus

In [ ]:
# Confrontare le inizializzazioni sul dataset con cluster ben separati
kmeans_random, kmeans_plus = compare_kmeans_init(X_blobs, n_clusters=4)

## Parte 3: Clustering Gerarchico

Implementiamo e applichiamo il clustering gerarchico ai nostri dataset.

### 3.1 Visualizzazione del dendrogramma

In [ ]:
def plot_dendrogram(X, max_samples=100, title="Dendrogramma del Clustering Gerarchico"):
    # Se il dataset è troppo grande, prendiamo un campione
    if X.shape[0] > max_samples:
        indices = np.random.choice(X.shape[0], max_samples, replace=False)
        X_sample = X[indices]
    else:
        X_sample = X
    
    # Calcolare la matrice di linkage
    linked = linkage(X_sample, method='ward')
    
    # Visualizzare il dendrogramma
    plt.figure(figsize=(12, 8))
    dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
    plt.title(title, fontsize=14)
    plt.xlabel('Campioni', fontsize=12)
    plt.ylabel('Distanza', fontsize=12)
    plt.axhline(y=15, c='k', linestyle='--', alpha=0.5)
    plt.text(X_sample.shape[0]/2, 16, 'Soglia di taglio suggerita', ha='center', va='center')
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizzare il dendrogramma per il dataset con cluster ben separati
plot_dendrogram(X_blobs, title="Dendrogramma per cluster ben separati")

### 3.2 Implementazione del Clustering Gerarchico

In [ ]:
def apply_hierarchical(X, n_clusters=4, linkage='ward', title="Clustering Gerarchico"):
    # Applicare il clustering gerarchico
    hierarchical = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    y_pred = hierarchical.fit_predict(X)
    
    # Visualizzare i risultati
    plt.figure(figsize=(10, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap='viridis', s=50, alpha=0.8)
    plt.title(f"{title} (linkage={linkage})", fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.colorbar(label='Cluster')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calcolare metriche di valutazione
    if n_clusters > 1:  # Silhouette score richiede almeno 2 cluster
        sil_score = silhouette_score(X, y_pred)
        ch_score = calinski_harabasz_score(X, y_pred)
        db_score = davies_bouldin_score(X, y_pred)
        
        print(f"Silhouette Score: {sil_score:.3f} (più alto è meglio)")
        print(f"Calinski-Harabasz Index: {ch_score:.3f} (più alto è meglio)")
        print(f"Davies-Bouldin Index: {db_score:.3f} (più basso è meglio)")
    
    return y_pred, hierarchical

In [ ]:
# Applicare il clustering gerarchico al dataset con cluster ben separati
y_pred_hier_blobs, hier_blobs = apply_hierarchical(X_blobs, n_clusters=4, title="Clustering Gerarchico su cluster ben separati")

In [ ]:
# Applicare il clustering gerarchico al dataset con cluster a forma di mezzaluna
y_pred_hier_moons, hier_moons = apply_hierarchical(X_moons, n_clusters=2, title="Clustering Gerarchico su cluster a forma di mezzaluna")

### 3.3 Confronto tra diversi metodi di linkage

In [ ]:
def compare_linkage_methods(X, n_clusters=4):
    linkage_methods = ['ward', 'complete', 'average', 'single']
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for i, method in enumerate(linkage_methods):
        # Applicare il clustering gerarchico
        hierarchical = AgglomerativeClustering(n_clusters=n_clusters, linkage=method)
        y_pred = hierarchical.fit_predict(X)
        
        # Calcolare silhouette score
        sil_score = silhouette_score(X, y_pred)
        
        # Visualizzare i risultati
        axes[i].scatter(X[:, 0], X[:, 1], c=y_pred, cmap='viridis', s=50, alpha=0.8)
        axes[i].set_title(f"Linkage: {method}\nSilhouette: {sil_score:.3f}", fontsize=14)
        axes[i].set_xlabel('Feature 1', fontsize=12)
        axes[i].set_ylabel('Feature 2', fontsize=12)
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Confrontare i metodi di linkage sul dataset con cluster ben separati
compare_linkage_methods(X_blobs, n_clusters=4)

In [ ]:
# Confrontare i metodi di linkage sul dataset con cluster a forma di mezzaluna
compare_linkage_methods(X_moons, n_clusters=2)

## Parte 4: DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

Implementiamo e applichiamo DBSCAN ai nostri dataset.

### 4.1 Determinare i parametri ottimali per DBSCAN

In [ ]:
def find_optimal_eps(X, min_samples=5, k=5):
    # Calcolare le distanze ai k vicini più prossimi
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    distances, indices = neigh.kneighbors(X)
    
    # Ordinare le distanze in ordine crescente
    distances = np.sort(distances[:, k-1])
    
    # Visualizzare il grafico delle distanze
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(distances)), distances, 'b-')
    plt.xlabel('Punti ordinati per distanza')
    plt.ylabel(f'Distanza al {k}-esimo vicino più prossimo')
    plt.title('Grafico delle distanze per determinare eps ottimale')
    plt.grid(True, alpha=0.3)
    
    # Calcolare la derivata per trovare il punto di massima curvatura
    derivative = np.gradient(distances)
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(derivative)), derivative, 'r-')
    plt.xlabel('Punti ordinati per distanza')
    plt.ylabel('Derivata della distanza')
    plt.title('Derivata delle distanze per identificare il "gomito"')
    plt.grid(True, alpha=0.3)
    
    # Trovare il punto di massima curvatura (approssimazione)
    knee_point = np.argmax(derivative) if len(derivative) > 0 else 0
    suggested_eps = distances[knee_point]
    
    print(f"Valore di eps suggerito: {suggested_eps:.3f}")
    print(f"Valore di min_samples suggerito: {min_samples}")
    
    return suggested_eps, min_samples

In [ ]:
# Trovare i parametri ottimali per DBSCAN sul dataset con cluster a forma di mezzaluna
eps_moons, min_samples_moons = find_optimal_eps(X_moons)

### 4.2 Implementazione di DBSCAN

In [ ]:
def apply_dbscan(X, eps=0.5, min_samples=5, title="DBSCAN Clustering"):
    # Applicare DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    y_pred = dbscan.fit_predict(X)
    
    # Contare il numero di cluster e punti di rumore
    n_clusters = len(set(y_pred)) - (1 if -1 in y_pred else 0)
    n_noise = list(y_pred).count(-1)
    
    # Visualizzare i risultati
    plt.figure(figsize=(10, 6))
    
    # Visualizzare i cluster
    unique_labels = set(y_pred)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))
    
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Punti di rumore in nero
            col = 'k'
        
        class_member_mask = (y_pred == k)
        xy = X[class_member_mask]
        plt.scatter(xy[:, 0], xy[:, 1], s=50, c=[col], alpha=0.8, label=f"Cluster {k}" if k != -1 else "Rumore")
    
    plt.title(f"{title}\neps={eps:.3f}, min_samples={min_samples}\nCluster trovati: {n_clusters}, Punti di rumore: {n_noise}", fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calcolare metriche di valutazione (solo se ci sono almeno 2 cluster e non tutti i punti sono rumore)
    if n_clusters >= 2 and n_noise < len(X):
        # Filtrare i punti di rumore per il calcolo delle metriche
        X_filtered = X[y_pred != -1]
        y_filtered = y_pred[y_pred != -1]
        
        if len(set(y_filtered)) >= 2:  # Almeno 2 cluster dopo il filtraggio
            sil_score = silhouette_score(X_filtered, y_filtered)
            ch_score = calinski_harabasz_score(X_filtered, y_filtered)
            db_score = davies_bouldin_score(X_filtered, y_filtered)
            
            print(f"Silhouette Score (esclusi punti di rumore): {sil_score:.3f} (più alto è meglio)")
            print(f"Calinski-Harabasz Index (esclusi punti di rumore): {ch_score:.3f} (più alto è meglio)")
            print(f"Davies-Bouldin Index (esclusi punti di rumore): {db_score:.3f} (più basso è meglio)")
    
    return y_pred, dbscan

In [ ]:
# Applicare DBSCAN al dataset con cluster a forma di mezzaluna
y_pred_dbscan_moons, dbscan_moons = apply_dbscan(X_moons, eps=eps_moons, min_samples=min_samples_moons, 
                                                title="DBSCAN su cluster a forma di mezzaluna")

In [ ]:
# Trovare i parametri ottimali per DBSCAN sul dataset con cluster concentrici
eps_circles, min_samples_circles = find_optimal_eps(X_circles)

# Applicare DBSCAN al dataset con cluster concentrici
y_pred_dbscan_circles, dbscan_circles = apply_dbscan(X_circles, eps=eps_circles, min_samples=min_samples_circles, 
                                                    title="DBSCAN su cluster concentrici")

### 4.3 Effetto dei parametri di DBSCAN

In [ ]:
def explore_dbscan_parameters(X, eps_values=[0.1, 0.3, 0.5], min_samples_values=[2, 5, 10]):
    fig, axes = plt.subplots(len(eps_values), len(min_samples_values), figsize=(15, 12))
    
    for i, eps in enumerate(eps_values):
        for j, min_samples in enumerate(min_samples_values):
            # Applicare DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            y_pred = dbscan.fit_predict(X)
            
            # Contare il numero di cluster e punti di rumore
            n_clusters = len(set(y_pred)) - (1 if -1 in y_pred else 0)
            n_noise = list(y_pred).count(-1)
            
            # Visualizzare i risultati
            ax = axes[i, j]
            
            # Visualizzare i cluster
            unique_labels = set(y_pred)
            colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))
            
            for k, col in zip(unique_labels, colors):
                if k == -1:
                    # Punti di rumore in nero
                    col = 'k'
                
                class_member_mask = (y_pred == k)
                xy = X[class_member_mask]
                ax.scatter(xy[:, 0], xy[:, 1], s=30, c=[col], alpha=0.8)
            
            ax.set_title(f"eps={eps:.2f}, min_samples={min_samples}\nCluster: {n_clusters}, Rumore: {n_noise}", fontsize=10)
            ax.set_xlabel('Feature 1', fontsize=8)
            ax.set_ylabel('Feature 2', fontsize=8)
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Esplorare l'effetto dei parametri di DBSCAN sul dataset con cluster a forma di mezzaluna
explore_dbscan_parameters(X_moons, 
                          eps_values=[0.1, 0.2, 0.3], 
                          min_samples_values=[3, 5, 10])

## Parte 5: Gaussian Mixture Models (GMM)

Implementiamo e applichiamo i Gaussian Mixture Models ai nostri dataset.

In [ ]:
def apply_gmm(X, n_components=4, title="Gaussian Mixture Model"):
    # Applicare GMM
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(X)
    y_pred = gmm.predict(X)
    
    # Visualizzare i risultati
    plt.figure(figsize=(10, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap='viridis', s=50, alpha=0.8)
    
    # Visualizzare le ellissi di confidenza
    for i in range(n_components):
        if not np.any(y_pred == i):
            continue
            
        mean = gmm.means_[i]
        covar = gmm.covariances_[i]
        v, w = np.linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180. * angle / np.pi  # Convertire in gradi
        
        # Disegnare l'ellisse
        ell = plt.matplotlib.patches.Ellipse(mean, v[0], v[1], 180. + angle, 
                                             edgecolor='black', facecolor='none', linewidth=2)
        plt.gca().add_patch(ell)
        plt.scatter(mean[0], mean[1], c='red', marker='X', s=200, alpha=1)
    
    plt.title(title, fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.colorbar(label='Cluster')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calcolare metriche di valutazione
    if n_components > 1:  # Silhouette score richiede almeno 2 cluster
        sil_score = silhouette_score(X, y_pred)
        ch_score = calinski_harabasz_score(X, y_pred)
        db_score = davies_bouldin_score(X, y_pred)
        
        print(f"Silhouette Score: {sil_score:.3f} (più alto è meglio)")
        print(f"Calinski-Harabasz Index: {ch_score:.3f} (più alto è meglio)")
        print(f"Davies-Bouldin Index: {db_score:.3f} (più basso è meglio)")
        print(f"Log-Likelihood: {gmm.score(X):.3f} (più alto è meglio)")
        print(f"BIC: {gmm.bic(X):.3f} (più basso è meglio)")
        print(f"AIC: {gmm.aic(X):.3f} (più basso è meglio)")
    
    return y_pred, gmm

In [ ]:
# Applicare GMM al dataset con cluster ben separati
y_pred_gmm_blobs, gmm_blobs = apply_gmm(X_blobs, n_components=4, title="GMM su cluster ben separati")

In [ ]:
# Applicare GMM al dataset con cluster a forma di mezzaluna
y_pred_gmm_moons, gmm_moons = apply_gmm(X_moons, n_components=2, title="GMM su cluster a forma di mezzaluna")

### 5.1 Determinare il numero ottimale di componenti per GMM

In [ ]:
def find_optimal_components(X, max_components=10):
    n_components_range = range(1, max_components + 1)
    bic = []
    aic = []
    silhouette = []
    
    for n_components in n_components_range:
        # Addestrare il GMM
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        gmm.fit(X)
        
        # Calcolare BIC e AIC
        bic.append(gmm.bic(X))
        aic.append(gmm.aic(X))
        
        # Calcolare silhouette score (solo per n_components >= 2)
        if n_components >= 2:
            y_pred = gmm.predict(X)
            silhouette.append(silhouette_score(X, y_pred))
        else:
            silhouette.append(0)  # Placeholder per n_components=1
    
    # Visualizzare i risultati
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(n_components_range, bic, 'o-', label='BIC')
    plt.plot(n_components_range, aic, 's-', label='AIC')
    plt.xlabel('Numero di componenti')
    plt.ylabel('Punteggio')
    plt.title('BIC e AIC per diversi numeri di componenti')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(n_components_range[1:], silhouette[1:], 'o-')
    plt.xlabel('Numero di componenti')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score per diversi numeri di componenti')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Trovare il numero ottimale di componenti
    optimal_bic = n_components_range[np.argmin(bic)]
    optimal_aic = n_components_range[np.argmin(aic)]
    optimal_silhouette = n_components_range[1:][np.argmax(silhouette[1:])]
    
    print(f"Numero ottimale di componenti secondo BIC: {optimal_bic}")
    print(f"Numero ottimale di componenti secondo AIC: {optimal_aic}")
    print(f"Numero ottimale di componenti secondo Silhouette: {optimal_silhouette}")
    
    return optimal_bic, optimal_aic, optimal_silhouette

In [ ]:
# Trovare il numero ottimale di componenti per il dataset con cluster ben separati
optimal_bic, optimal_aic, optimal_silhouette = find_optimal_components(X_blobs, max_components=10)

## Parte 6: Confronto tra Algoritmi di Clustering

Confrontiamo le prestazioni dei diversi algoritmi di clustering sui nostri dataset.

In [ ]:
def compare_clustering_algorithms(X, true_labels=None, n_clusters=4):
    # Applicare i diversi algoritmi
    algorithms = {
        'K-means': KMeans(n_clusters=n_clusters, random_state=42, n_init=10),
        'Hierarchical (Ward)': AgglomerativeClustering(n_clusters=n_clusters, linkage='ward'),
        'DBSCAN': DBSCAN(eps=0.3, min_samples=5),
        'GMM': GaussianMixture(n_components=n_clusters, random_state=42)
    }
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    results = {}
    
    for i, (name, algorithm) in enumerate(algorithms.items()):
        # Addestrare l'algoritmo
        if name == 'GMM':
            algorithm.fit(X)
            y_pred = algorithm.predict(X)
        else:
            y_pred = algorithm.fit_predict(X)
        
        # Gestire il caso di DBSCAN che può avere etichette -1 (rumore)
        if name == 'DBSCAN':
            n_clusters_found = len(set(y_pred)) - (1 if -1 in y_pred else 0)
            n_noise = list(y_pred).count(-1)
            title = f"{name}\nCluster trovati: {n_clusters_found}, Rumore: {n_noise}"
        else:
            title = name
        
        # Visualizzare i risultati
        axes[i].scatter(X[:, 0], X[:, 1], c=y_pred, cmap='viridis', s=50, alpha=0.8)
        axes[i].set_title(title, fontsize=14)
        axes[i].set_xlabel('Feature 1', fontsize=12)
        axes[i].set_ylabel('Feature 2', fontsize=12)
        axes[i].grid(True, alpha=0.3)
        
        # Calcolare metriche di valutazione
        metrics = {}
        
        # Per DBSCAN, calcolare le metriche solo sui punti non di rumore
        if name == 'DBSCAN' and -1 in y_pred:
            X_filtered = X[y_pred != -1]
            y_filtered = y_pred[y_pred != -1]
            
            if len(set(y_filtered)) >= 2 and len(y_filtered) > 0:  # Almeno 2 cluster dopo il filtraggio
                metrics['silhouette'] = silhouette_score(X_filtered, y_filtered)
                metrics['calinski_harabasz'] = calinski_harabasz_score(X_filtered, y_filtered)
                metrics['davies_bouldin'] = davies_bouldin_score(X_filtered, y_filtered)
        elif len(set(y_pred)) >= 2:  # Almeno 2 cluster
            metrics['silhouette'] = silhouette_score(X, y_pred)
            metrics['calinski_harabasz'] = calinski_harabasz_score(X, y_pred)
            metrics['davies_bouldin'] = davies_bouldin_score(X, y_pred)
        
        results[name] = {'labels': y_pred, 'metrics': metrics}
    
    plt.tight_layout()
    plt.show()
    
    # Visualizzare le metriche di valutazione
    metrics_df = pd.DataFrame()
    for name, result in results.items():
        if result['metrics']:
            metrics_df[name] = pd.Series(result['metrics'])
    
    if not metrics_df.empty:
        print("Metriche di valutazione:")
        print(metrics_df.T)
    
    return results

In [ ]:
# Confrontare gli algoritmi sul dataset con cluster ben separati
results_blobs = compare_clustering_algorithms(X_blobs, true_labels=y_blobs, n_clusters=4)

In [ ]:
# Confrontare gli algoritmi sul dataset con cluster a forma di mezzaluna
results_moons = compare_clustering_algorithms(X_moons, true_labels=y_moons, n_clusters=2)

In [ ]:
# Confrontare gli algoritmi sul dataset con cluster concentrici
results_circles = compare_clustering_algorithms(X_circles, true_labels=y_circles, n_clusters=2)

## Parte 7: Caso di Studio - Segmentazione dei Clienti

Applichiamo gli algoritmi di clustering a un caso di studio reale: la segmentazione dei clienti.

In [ ]:
# Caricare il dataset Mall Customer Segmentation
url = "https://raw.githubusercontent.com/SteffiPeTaffy/machineLearningAZ/master/Machine%20Learning%20A-Z%20Template%20Folder/Part%204%20-%20Clustering/Section%2025%20-%20Hierarchical%20Clustering/Mall_Customers.csv"
customers = pd.read_csv(url)
customers.head()

In [ ]:
# Esplorare il dataset
print(f"Dimensioni del dataset: {customers.shape}")
print("\nInformazioni sul dataset:")
customers.info()
print("\nStatistiche descrittive:")
customers.describe()

In [ ]:
# Visualizzare la distribuzione delle variabili
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.histplot(customers['Age'], kde=True)
plt.title('Distribuzione dell\'età')

plt.subplot(2, 2, 2)
sns.countplot(x='Gender', data=customers)
plt.title('Distribuzione del genere')

plt.subplot(2, 2, 3)
sns.histplot(customers['Annual Income (k$)'], kde=True)
plt.title('Distribuzione del reddito annuale')

plt.subplot(2, 2, 4)
sns.histplot(customers['Spending Score (1-100)'], kde=True)
plt.title('Distribuzione del punteggio di spesa')

plt.tight_layout()
plt.show()

In [ ]:
# Preparare i dati per il clustering
# Utilizziamo solo reddito annuale e punteggio di spesa per la segmentazione
X_customers = customers[['Annual Income (k$)', 'Spending Score (1-100)']].values

# Visualizzare i dati
plt.figure(figsize=(10, 6))
plt.scatter(X_customers[:, 0], X_customers[:, 1], s=50, alpha=0.8)
plt.title('Reddito annuale vs Punteggio di spesa', fontsize=14)
plt.xlabel('Reddito annuale (k$)', fontsize=12)
plt.ylabel('Punteggio di spesa (1-100)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Determinare il numero ottimale di cluster con il metodo del gomito
distortions_customers, silhouette_customers = plot_elbow_method(X_customers, max_k=10)

In [ ]:
# Applicare K-means con il numero ottimale di cluster
optimal_k = 5  # Basato sul metodo del gomito e silhouette score
y_pred_customers, centers_customers = apply_kmeans(X_customers, n_clusters=optimal_k, 
                                                  title="Segmentazione dei clienti con K-means")

In [ ]:
# Interpretare i cluster
customers_clustered = customers.copy()
customers_clustered['Cluster'] = y_pred_customers

# Statistiche per cluster
cluster_stats = customers_clustered.groupby('Cluster').agg({
    'Age': ['mean', 'min', 'max'],
    'Annual Income (k$)': ['mean', 'min', 'max'],
    'Spending Score (1-100)': ['mean', 'min', 'max'],
    'CustomerID': 'count'
}).rename(columns={'CustomerID': 'Count'})

print("Statistiche per cluster:")
display(cluster_stats)

In [ ]:
# Visualizzare i cluster con etichette interpretative
plt.figure(figsize=(12, 8))
colors = ['red', 'blue', 'green', 'cyan', 'magenta']
cluster_labels = [
    'Reddito basso, Spesa bassa',
    'Reddito alto, Spesa bassa',
    'Reddito basso, Spesa alta',
    'Reddito alto, Spesa alta',
    'Reddito medio, Spesa media'
]

for i in range(optimal_k):
    plt.scatter(X_customers[y_pred_customers == i, 0], X_customers[y_pred_customers == i, 1], 
                s=100, c=colors[i], label=f'Cluster {i+1}: {cluster_labels[i]}')

plt.scatter(centers_customers[:, 0], centers_customers[:, 1], s=300, c='yellow', marker='*', label='Centroidi')
plt.title('Segmentazione dei clienti', fontsize=16)
plt.xlabel('Reddito annuale (k$)', fontsize=14)
plt.ylabel('Punteggio di spesa (1-100)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizzare la distribuzione dell'età per cluster
plt.figure(figsize=(12, 6))
sns.boxplot(x='Cluster', y='Age', data=customers_clustered)
plt.title('Distribuzione dell\'età per cluster', fontsize=14)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Età', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizzare la distribuzione del genere per cluster
gender_cluster = pd.crosstab(customers_clustered['Cluster'], customers_clustered['Gender'])
gender_cluster.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Distribuzione del genere per cluster', fontsize=14)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Conteggio', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(title='Genere')
plt.tight_layout()
plt.show()

## Conclusioni

In questa esercitazione pratica, abbiamo esplorato diversi algoritmi di clustering e li abbiamo applicati a vari dataset, incluso un caso di studio reale sulla segmentazione dei clienti. Abbiamo visto come:

1. **K-means** è efficace per cluster ben separati e di forma sferica, ma ha difficoltà con forme complesse.
2. **Clustering Gerarchico** offre una rappresentazione gerarchica dei dati e può essere utile per esplorare la struttura dei dati.
3. **DBSCAN** eccelle nell'identificare cluster di forma arbitraria e nel rilevare outlier.
4. **Gaussian Mixture Models** forniscono un approccio probabilistico al clustering e sono flessibili nella forma dei cluster.

Abbiamo anche imparato l'importanza di:
- Scegliere il numero appropriato di cluster
- Selezionare l'algoritmo giusto in base alla natura dei dati
- Valutare la qualità dei cluster con metriche appropriate
- Interpretare i risultati nel contesto del problema specifico

Nel caso di studio sulla segmentazione dei clienti, abbiamo identificato cinque segmenti distinti di clienti in base al loro reddito e comportamento di spesa, fornendo informazioni preziose per strategie di marketing mirate.

## Esercizi Aggiuntivi

1. Prova a segmentare i clienti utilizzando tutte e tre le variabili numeriche (età, reddito, punteggio di spesa). Come cambiano i cluster?
2. Applica DBSCAN al dataset di segmentazione dei clienti. Quali parametri sceglieresti e perché?
3. Implementa il clustering gerarchico sul dataset di segmentazione dei clienti e confronta i risultati con K-means.
4. Crea un nuovo dataset sintetico con cluster di forme complesse e confronta le prestazioni dei diversi algoritmi.
5. Implementa una versione semplificata di K-means da zero (senza utilizzare scikit-learn) e confronta i risultati con l'implementazione di scikit-learn.